## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append("../")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [20]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Monte Patria,CL,-30.6919,-70.9467,57.36,69,0,0.65,clear sky
1,1,Hermanus,ZA,-34.4187,19.2345,52.90,94,62,6.91,broken clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,46.06,76,40,0.00,scattered clouds
3,3,Hasaki,JP,35.7333,140.8333,97.29,71,0,8.77,clear sky
4,4,Ponta do Sol,PT,32.6667,-17.1000,74.03,82,1,3.65,clear sky


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Choose the minimum temperature for your preferred destination."))
max_temp = float(input("Choose the maximum temperature for your preferred destination."))

Choose the minimum temperature for your preferred destination.75
Choose the maximum temperature for your preferred destination.90


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,George Town,MY,5.4112,100.3354,87.89,67,20,1.01,few clouds
12,12,Cabo San Lucas,MX,22.8909,-109.9124,83.61,72,88,6.69,overcast clouds
15,15,Waingapu,ID,-9.6567,120.2641,86.79,55,18,18.84,few clouds
17,17,San Cristobal,VE,7.7669,-72.2250,75.38,97,88,4.29,overcast clouds
26,26,Marawi,PH,7.9986,124.2928,85.50,52,99,4.34,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
647,647,Taytayan,PH,7.7339,126.5019,87.04,69,82,8.12,broken clouds
650,650,Bahía Solano,CO,6.2262,-77.4044,78.75,85,100,1.95,overcast clouds
651,651,Guangyuan,CN,32.4420,105.8230,82.17,62,99,6.31,overcast clouds
653,653,Guiyang,CN,26.5833,106.7167,84.69,48,20,8.95,few clouds


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                218
City                   218
Country                218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                218
City                   218
Country                218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,George Town,MY,87.89,few clouds,5.4112,100.3354,
12,Cabo San Lucas,MX,83.61,overcast clouds,22.8909,-109.9124,
15,Waingapu,ID,86.79,few clouds,-9.6567,120.2641,
17,San Cristobal,VE,75.38,overcast clouds,7.7669,-72.2250,
26,Marawi,PH,85.50,overcast clouds,7.9986,124.2928,
27,Kahului,US,82.42,clear sky,20.8947,-156.4700,
28,Phek,IN,77.77,scattered clouds,25.6667,94.5000,
35,Matara,LK,82.06,overcast clouds,5.9485,80.5353,
39,Khandyga,RU,80.06,scattered clouds,62.6667,135.6000,
40,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [6]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
hotel_df.count()
hotel_df.head(10)

NameError: name 'hotel_df' is not defined

In [3]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'hotel_df' is not defined

In [4]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [5]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

NameError: name 'hotel_df' is not defined